In [2]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)

2.2.0


In [4]:
output_dir = "nmt_pure_transformer"
en_vocab_file = os.path.join(output_dir, "en_vocab")
zh_vocab_file = os.path.join(output_dir, "zh_vocab")
checkpoint_path = os.path.join(output_dir, "checkpoints")
log_dir = os.path.join(output_dir, 'logs')
download_dir = "tensorflow-datasets/downloads"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Outline

### Data

1. Prepare the datasets (tf.data.Dataset)
2. Build the dictionary from en, zh datasets (by SubwordTextEncoder)
    1. 中文需定義 `max_subword_length=1`
3. Preprocessing
    1. 定義一個 `encode` function 負責加入 BOS, EOS 的 index 到 subword encoding 當中
    2. 去掉長度為 40 的 data
    3. 利用 `padded_batch` 將 data 全部都 padding 成一樣長度

### Transformer

1. Padding mask
2. Look Ahead mask
3. Feed-forward network
4. Multi-Head Attention network
5. Encoder layer
    1. Sublayer
        1. MHA (padding mask)
        2. FFN
    2. Dropout
    3. Residual connection
    4. LayerNorm
6. Decoder layer
    1. Sublayer
        1. Masked MHA (padding + look ahead mask)
        2. Encoder-Decoder MHA (padding mask)
            1. Masked MHA = Q
            2. Encoder = K, V
        3. FFN
    2. Dropout
    3. Residual connection
    4. LayerNorm
7. Positional Encoding
8. Encoder
    1. Embedding
    2. Positional Encoding
    3. Regularization
    4. Encoder layer
9. Decoder
    1. Embedding
    2. Positional Encoding
    3. Regularization
    4. Decoder layer
10. Final linear layer

### Training

1. inp
2. tar_inp = tar[:, :-1]
3. tar_real = tar[:, 1:]
4. inp + tar_int = pred
5. cross_entropy_loss(pred, tar_real)
